TEMA: MERCADO DE OPÇÕES

BRUNO  SILVA RA 091585

DENISE LEITE RA 183137

MILENA ROCHA RA 183146

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## OBJETIVO

O objetivo deste trabalho é criar um modelo de regressão capaz de prever o preço de uma opção no dia seguinte. Teremos como base os valores históricos de movimentação do mercado financeiro tanto da ação, quanto da opção a ela vinculada. O usuário informará o código da ação (papel) e em seguida o modelo apresentará a chance de acerto e o valor previsto.

## O MERCADO DE OPÇÕES

> No mercado de opções, negocia-se o direito de comprar ou de vender um bem por um preço fixo numa data futura. 
Quem adquirir o direito deve pagar um prêmio ao vendedor. Este prêmio não é o preço do bem, mas apenas um valor pago para ter a opção (possibilidade) de comprar ou vender o referido bem em uma data futura por um preço previamente acordado.

> O objeto de negociação pode ser um ativo financeiro ou uma mercadoria, negociados em pregão, com ampla transparência. O comprador da opção, também chamado titular, sempre terá o direito do exercício, mas não obrigação de exercê-lo. O vendedor da opção, também chamado lançador, terá a obrigação de atender ao exercício caso o titular opte por exercer seu direito.

##### Fonte: https://www.investidor.gov.br/menu/Menu_Investidor/derivativos/mercado_opcoes.html

### GLOSSÁRIO DO MERCADO DE OPÇÕES

#### Opção de Compra ou Call

> Vendedor é o lançador e possui a obrigação de vender o ativo-objeto no preço predeterminado no vencimento acordado, recebeu um prêmio por ter esta obrigação.

> E o comprador que é o titular e pagou o prêmio adquire a opcão de comprar por um preço combinado que é o strike em um prazo combinado por ambos.

#### Opção de Venda ou Put

> Vendedor é o títular já que pagou o prêmio para ter o direito, mas não a obrigação, de vender e o comprador que é o lançador tem a obrigação de comprar por um preço combinado que é o strike em um prazo combinado por ambos.

#### Ativo-objeto

> O ativo-objeto é o bem de referência de um contrato de opção. O exemplo mais comum é um lote de ações. O ativo de referência apenas é negociado se o titular exercer o seu direito conforme combinado.

#### Preço de exercício (strike)

> É o preço combinado do ativo-objeto do contrato de opção.

#### Data de vencimento
> É o dia da expiração do contrato. Neste dia, o contrato de opção perde o seu valor de mercado, ou como dizem, a opção vira pó. 
> No mercado BM&F, a data ocorre sempre na quarta-feira mais próxima do 15º dia de cada mês para opções de Índice. Já no Bovespa, a expiração acontece sempre na 3º segunda-feira de cada mês.

> Existem duas formas de exercer direito de acordo: durante ou apenas na data de vencimento.
No Brasil, a prática mais comum é utilizar a opção americana para as Calls e europeia para as Puts.

#### Titular ou comprador da opção
> O titular de uma opção é o investidor que compra um contrato de opção no mercado, adquirindo para si o direito de negociar um ativo-objeto pelo preço de exercício (strike), durante determinado período de tempo ou em uma data pré-determinada.
> Ele tem o benefício de operar alavancado, mas corre o risco de perder todo o capital investido.

#### Lançador ou vendedor da opção
> O lançador de uma opção é o investidor que vende um contrato de opção no mercado, adquirindo para si a obrigação de negociar um ativo-objeto pelo preço de exercício (strike), durante determinado período de tempo ou em uma data pré-determinada.
> O vendedor no mercado de opções está sempre especulando, seja vendendo opções de compra ou de venda. A sua confiança é que a tendência do mercado se confirme e o benefício do prêmio não seja exercido. 

#### Prêmio
> Prêmio é o preço pago pelo titular de um contrato de opção ao vendedor no momento da aquisição da opção.

##### Fonte: https://blog.rico.com.vc/mercado-de-opcoes

### DEFINIÇÕES INICIAIS

Tecnicamente, as primeiras definições efetuadas foram as bibliotecas que utilizaríamos para a análise e exploração dos dados e para a limpeza/pré-processamento do Dataset. Utilizamos as bibliotecas que aprendemos em aula, como também a biblioteca de criação de gráficos.

Posteriormente, outras bibliotecas foram adicionadas, como as definições de dataframe, que utilizamos para exibição dos dados do layout, e as bibliotecas de separação em treino e teste e dos modelos aplicados.

In [ ]:
#Importando as Bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as sm
from IPython.display import HTML
import xgboost as xgb
from sklearn.metrics import explained_variance_score
from scipy.stats import pearsonr

Definimos um estilo de DataFrame para facilitar a visualização dos dados descritivos do layout.

In [ ]:
#Definindo o estilo - DataFrame Layout
styles = [
    dict(selector="td", props=[('font-size', '13px'),
                               ("text-align", "left")]),
    dict(selector="th", props=[("font-size", "13px"),
                               ("text-align", "center")]),
]

Então, importamos o arquivo contendo csv, já definindo a coluna indíce.

In [ ]:
#Importando o arquivo
file = "/kaggle/input/b3-stock-quotes/COTAHIST_A2009_to_A2018P.csv"
df = pd.read_csv(file,index_col=0)
df.head(2)

## Exploração dos Dados

Utilizaremos um dataset que contém o histórico de preços dos títulos negociados na bolsa de valores de São Paulo (BMFBOVESPA), nos períodos de 2009 a 2018).

As cotações são fornecidas na moeda e forma de cotação da época, sem nenhum ajuste para a inflação ou proventos (dividendos, bonificações, direitos de subscrição, etc.) distribuídos pelas empresas emissoras.

##### Fonte: http://www.bmfbovespa.com.br/pt_br/servicos/market-data/historico/mercado-a-vista/cotacoes-historicas/
##### Fonte: https://www.kaggle.com/gbonesso/b3-stock-quotes

### Definição Linhas e colunas do dataset

A primeira etapa realizada foi de verificar quantas linhas e quantas colunas existem no dataset.

In [ ]:
#linhas e colunas
n_lin = len(df.index)
n_col = len(df.columns)

print('Linhas',n_lin,'Colunas', n_col)

### Definição do Layout do Arquivo

Após a verificação da quantidade de linhas e colunas, o próximo passo efetuado foi localizar quais informações estavam disponíveis no arquivo. A página das séries históricas da Bovespa possui um documento com as informações dos arquivos disponiblizados. A partir desta informação, e pela visualização de uma parcela dos dados, classificamos os tipos de dados do dataset, conforme mostra a tabela abaixo.

##### Fonte: SeriesHistoricas_Layout.pdf, 
##### disponível em http://www.bmfbovespa.com.br/pt_br/servicos/market-data/historico/mercado-a-vista/cotacoes-historicas/

In [ ]:
VARIAVEIS = [["TIPREG","Quantitativa Discreta","TIPO DE REGISTRO - FIXO “01”"],["DATPRE","Quantitativa Discreta", "Data de realização do pregão FORMATO “AAAAMMDD”"],
             ["CDOBDI", "Qualitativa Nominal", "CÓDIGO BDI - UTILIZADO PARA CLASSIFICAR OS PAPÉIS NA EMISSÃO DO BOLETIM DIÁRIO DE INFORMAÇÕES"],
             ["CODNEG", "Qualitativa Nominal", "CÓDIGO DE NEGOCIAÇÃO DO PAPEL"],["TPMERC","Qualitativa Nominal", "CÓD. DO MERCADO EM QUE O PAPEL ESTÁ CADASTRADO"],
             ["NOMRES","Qualitativa Nominal", "NOME RESUMIDO DA EMPRESA EMISSORA DO PAPEL"],["ESPECI", "Qualitativa Nominal", "ESPECIFICAÇÃO DO PAPEL"],
             ["PRAZOT","Quantitativa Discreta", "PRAZO EM DIAS DO MERCADO A TERMO"],["MODREF","Qualitativa Nominal", "MOEDA DE REFERÊNCIA - MOEDA USADA NA DATA DO PREGÃO"],
             ["PREABE", "Quantitativa Contínua", "PREÇO DE ABERTURA DO PAPELMERCADO NO PREGÃO"],["PREMAX","Quantitativa Contínua", "PREÇO MÁXIMO DO PAPELMERCADO NO PREGÃO"],
             ["PREMIN","Quantitativa Contínua", "PREÇO MÍNIMO DO PAPEL- MERCADO NO PREGÃO"],["PREMED", "Quantitativa Contínua", "PREÇO MÉDIO DO PAPEL- MERCADO NO PREGÃO"],
             ["PREULT","Quantitativa Contínua", "PREÇO DO ÚLTIMO NEGÓCIO DO PAPEL-MERCADO NO PREGÃO"],["PREOFC","Quantitativa Contínua", "PREÇO DA MELHOR OFERTA DE COMPRA DO PAPEL MERCADO"],
             ["PREOFV", "Quantitativa Contínua", "PREÇO DA MELHOR OFERTA DE VENDA DO PAPEL MERCADO"],["TOTNEG","Quantitativa Discreta", "NÚMERO DE NEGÓCIOS EFETUADOS COM O PAPEL- MERCADO NO PREGÃO"],
             ["QUATOT","Quantitativa Discreta", "QUANTIDADE TOTAL DE TÍTULOS NEGOCIADOS NESTE PAPEL- MERCADO"],["VOLTOT", "Quantitativa Discreta", "VOLUME TOTAL DE TÍTULOS NEGOCIADOS NESTE PAPEL- MERCADO"],
             ["PREEXE","Quantitativa Contínua", "PREÇO DE EXERCÍCIO PARA O MERCADO DE OPÇÕES OU VALOR DO CONTRATO PARA O MERCADO DE TERMO SECUNDÁRIO"],
             ["INDOPC","Qualitativa Nominal", "INDICADOR DE CORREÇÃO DE PREÇOS DE EXERCÍCIOS OU VALORES DE CONTRATO PARA OS MERCADOS DE OPÇÕES OU TERMO SECUNDÁRIO"],
             ["DATVEN", "Quantitativa Discreta", "DATA DO VENCIMENTO PARA OS MERCADOS DE OPÇÕES OU TERMO SECUNDÁRIO - FORMATO “AAAAMMDD”"],
             ["FATCOT", "Quantitativa Discreta", "FATOR DE COTAÇÃO DO PAPEL - ‘1’ = COTAÇÃO UNITÁRIA/‘1000’ = COTAÇÃO POR LOTE DE MIL AÇÕES"],
             ["PTOEXE","Quantitativa Discreta", "PREÇO DE EXERCÍCIO EM PONTOS PARA OPÇÕES REFERENCIADAS EM DÓLAR OU VALOR DE CONTRATO EM PONTOS PARA TERMO SECUNDÁRIO - PARA OS REFERENCIADOS EM DÓLAR, CADA PONTO EQUIVALE AO VALOR, NA MOEDA CORRENTE, DE UM CENTÉSIMO DA TAXA MÉDIA DO DÓLAR COMERCIAL INTERBANCÁRIO DE FECHAMENTO DO DIA ANTERIOR, OU SEJA, 1 PONTO = 1/100 US$"],
             ["CODISI", "Qualitativa Nominal", "CÓDIGO DO PAPEL NO SISTEMA ISIN OU CÓDIGO INTERNO DO PAPEL - CÓDIGO DO PAPEL NO SISTEMA ISIN A PARTIR DE 15-05-1995"],
             ["DISMES","Quantitativa Discreta", "NÚMERO DE DISTRIBUIÇÃO DO PAPEL - NÚMERO DE SEQÜÊNCIA DO PAPEL CORRESPONDENTE AO ESTADO DE DIREITO VIGENTE"]] 
DF_VARIAV = pd.DataFrame(VARIAVEIS, columns=["Variavel", "Classificação", "Descrição"]).style.set_table_styles(styles).hide_index()
DF_VARIAV

### RELAÇÃO DOS VALORES PARA TIPO DE MERCADO 

O código de tipo de mercado indica a que tipo de mercado aquela operação pertence. No arquivo de layout disponibilizado pela B3, existem 10 códigos distintos relacionados. Verificamos,então, quais os tipos de mercados existentes do dataset que utilizaremos. Apenas as transações de mercado futuro (FUTURO COM RETENÇÃO DE GANHO e FUTURO COM MOVIMENTAÇÃO CONTÍNUA) não estavam contidos no dataset.



In [ ]:
#Tipos de Mercado existentes no arquivo
tpMercado = df [['TPMERC']]
tpMercado = np.unique(tpMercado)
tpMercado

In [ ]:
DESC_MERC = [["010","VISTA"],
            ["012","EXERCÍCIO DE OPÇÕES DE COMPRA"],
            ["013", "EXERCÍCIO DE OPÇÕES DE VENDA"],
            ["017","LEILÃO"],
            ["020","FRACIONÁRIO"],
            ["030", "TERMO"],
            ["070", "OPÇÕES DE COMPRA"],
            ["080", "OPÇÕES DE VENDA"]]

DF_MERC = pd.DataFrame(DESC_MERC, columns=["Código", "Descrição"]).style.set_table_styles(styles).hide_index()
DF_MERC

###  RELAÇÃO DOS CÓDIGOS BDI

O código BDI é o código utilizado para classificar os papéis na emissão do boletim diário de informações.
No arquivo de layout disponilizado pela B3, existem 43 códigos distintos de classificação do papel. 
Desta forma, verificamos quais os códigos existentes no dataset utilizado. Dos 43 existentes, nosso dataset possui 28, confome descrito abaixo.

In [ ]:
codBDI = df[['CODBDI']]
codBDI = np.unique(codBDI)
codBDI

In [ ]:
DESC_DBI = [["02","LOTE PADRAO"],
            ["05","SANCIONADAS PELOS REGULAMENTOS BMFBOVESPA"],
            ["06", "CONCORDATARIAS"],
            ["07","RECUPERACAO EXTRAJUDICIAL"],
            ["08","RECUPERAÇÃO JUDICIAL"],
            ["10", "DIREITOS E RECIBOS"],
            ["12", "FUNDOS IMOBILIARIOS"],
            ["14", "CERT.INVEST/TIT.DIV.PUBLICA"],
            ["22", "BÔNUS (PRIVADOS)"],
            ["32", "EXERCICIO DE OPCOES DE COMPRA DE INDICES"],
            ["33", "EXERCICIO DE OPCOES DE VENDA DE INDICES"],
            ["38", "EXERCICIO DE OPCOES DE COMPRA"],
            ["42", "EXERCICIO DE OPCOES DE VENDA"],
            ["46", "LEILAO DE NAO COTADOS"],
            ["48", "LEILAO DE PRIVATIZACAO"],
            ["49", "LEILAO DO FUNDO RECUPERACAO ECONOMICA ESPIRITO SANTO"],
            ["50", "LEILAO"],
            ["51", "LEILAO FINOR"],
            ["52", "LEILAO FINAM"],
            ["58", "OUTROS"],
            ["62", "MERCADO A TERMO"],
            ["74", "OPCOES DE COMPRA DE INDICES"],
            ["75", "OPCOES DE VENDA DE INDICES"],
            ["78", "OPCOES DE COMPRA"],
            ["82", "OPCOES DE VENDA"],
            ["96", "MERCADO FRACIONARIO"]]

DF_DBI = pd.DataFrame(DESC_DBI, columns=["Código", "Descrição"]).style.set_table_styles(styles).hide_index()
DF_DBI

### RELAÇÃO DE VALORES PARA ESPECIFICAÇÃO

Este campo relaciona a especificação do papel da negociação. Ao relacionar este campos, pudemos perceber que, no layout disponibilizados existem cerca de 80 códigos, e no arquivo dataset existem 466 códigos distintos. Não encontramos esses códigos relacionados.

In [ ]:
codESPECI = df [['ESPECI']]
codESPECI = np.unique(codESPECI)
codESPECI = pd.DataFrame(codESPECI).count()
codESPECI

### AS CINCO AÇÕES MAIS COTADAS NO MERCADO DE OPÇÕES

Como o foco deste trabalho se destina ao mercado de opções, a primeira pergunta relacionada que nos foi colocada foi quais as ações que teriam o maior destaque, ou seja, quais as ações mais cotadas (em volume de transações). Abaixo, o gráfico demonstra as cinco ações mais cotadas de todo o período.

In [ ]:
#Quais as 5 ações mais negociadas no mercado de opções? 
stg = df[['CODNEG','TPMERC']]
mask = ((stg['TPMERC'] == 12) | (stg['TPMERC'] == 13) | (stg['TPMERC'] == 70) | (stg['TPMERC'] == 80))

by_cod = stg[mask]
by_cod = by_cod.groupby('CODNEG').size().sort_values(ascending=False)
by_cod = by_cod.head(5)

by_cod.plot(kind = 'barh', color='gray',figsize=(9,8),grid = False)

plt.title('AS 5 AÇÕES MAIS NEGOCIADAS NO MERCADO DE OPÇÕES')
plt.xlabel('TRANSAÇÕES')
plt.ylabel('AÇÕES')
plt.show()

A partir dessas 5 ações mais cotadas, damos a chance de escolha para prever o preço da opção de uma delas.

### SELECIONE UMA DAS CINCO AÇÕES PARA O MODELO:

#### <li> 1 - PETRA20 </li>  
#### <li> 2 - PETRC20 </li>
#### <li> 3 - PETRL22 </li>
#### <li> 4 - PETRK20 </li>
#### <li> 5 - VALEL36 </li>

In [ ]:
print('Selecione a ação que deseja receber o valor de amanhã:')
print('1 - PETRA20, 2 - PETRC20, 3 - PETRL22, 4 - PETRK20 ou 5 - VALEL36')

Acao = int(input('Insira o Codigo: '))

if Acao == 1:
    Acao = 'PETRA20'
elif Acao == 2:
    Acao = 'PETRC20'
elif Acao == 3:
    Acao = 'PETRL22'
elif Acao == 4:
    Acao = 'PETRK20'
elif Acao == 5:
    Acao = 'VALEL36'
else:
    Acao = 'PETRA20'
    
print('A ação selecionada foi:', Acao)

### NÚMERO DE TRANSAÇÕES (POR ANO)

Considerando a ação selecionada acima, analisamos o número de transações do período, anualmente.

In [ ]:
    #Número de Transações por ano
    stg = df[['CODNEG','DATPRE','TPMERC']]
    stg['ANO'] = pd.to_datetime(stg.DATPRE).dt.year
    mask = ((stg['TPMERC'] == 12) | (stg['TPMERC'] == 13) | (stg['TPMERC'] == 70) | (stg['TPMERC'] == 80))

    by_year = stg[mask]
    mask1 = (stg['CODNEG'] == Acao)
    by_year = by_year[mask1]
    by_year = by_year.groupby(['ANO']).size()

    by_year.plot(kind = 'bar', color='green',figsize=(9,10),grid = False)

    plt.xticks(rotation='90')
    plt.title('NÚMERO DE TRANSAÇÕES POR ANO - '+ Acao)
    plt.xlabel('ANO')
    plt.ylabel('QUANTIDADE')
    #print(by_year)
    plt.show()

### A OPERAÇÃO MAIS COMUM (NO MERCADO DE OPÇÕES)

Verifcamos qual é a operação do mercado que opções que mais ocorre a ação selecionada. 

In [ ]:
#Mercados
#012 EXERCÍCIO DE OPÇÕES DE COMPRA
#013 EXERCÍCIO DE OPÇÕES DE VENDA
#070 OPÇÕES DE COMPRA
#080 OPÇÕES DE VENDA

    stg = df[['CODNEG','TPMERC']]
    mask = ((stg['TPMERC'] == 12) | (stg['TPMERC'] == 13) | (stg['TPMERC'] == 70) | (stg['TPMERC'] == 80))

    by_cod = stg[mask]
    mask1 = (stg['CODNEG'] == Acao)
    by_market = by_cod[mask1]

    by_market = by_market.groupby(['TPMERC']).size()
    
    print(by_market)

### PREÇO MÉDIO DE EXERCÍCIO

In [ ]:
    stg = df[['CODNEG','TPMERC','PREEXE','DATPRE']]
    mask = ((stg['TPMERC'] == 12) | (stg['TPMERC'] == 13) | (stg['TPMERC'] == 70) | (stg['TPMERC'] == 80))
    stg['ANO'] = pd.to_datetime(stg.DATPRE).dt.year

    by_price = stg[mask]
    mask1 = (stg['CODNEG']== Acao)
    by_price = by_price[mask1]
    by_price = by_price.groupby('ANO')['PREEXE'].mean().round(2)

    by_price.plot(color = 'purple',figsize=(12,5),grid = True)

    plt.xlabel('ANO DE REFERÊNCIA')
    plt.ylabel('PREÇO MÉDIO DE EXERICIO')
    plt.show()

### PRIMEIROS VALORES NEGOCIADOS

In [ ]:
df_filter = df[df.CODNEG == Acao]
df_filter = df_filter[(df_filter['DATPRE'] >= '2009-01-01') & (df_filter['DATPRE'] <= '2011-02-01')]
df_filter.head(5)

### MOEDA UTILIZADA NO PERÍODO

A única moeda utilizada no pregão é o Real Brasileiro

In [ ]:
#Moeda utilizada no pregão
moedaMerc = df[['MODREF']]
moedaMerc = np.unique(moedaMerc)
moedaMerc

### TAXA DE CORREÇÃO DOS CONTRATOS NOS ARQUIVOS


Não houve taxa de correção dos contratos

In [ ]:
#Taxa de correção dos contratos nos arquivos.
indCorr = df[['INDOPC']]
indCorr = np.unique(indCorr)
indCorr

### FATOR DE COTAÇÃO DO MERCADO DE OPÇÕES

O fator de cotação define o volume (quantidade) do papel cotado. 

1 representa uma cotação de uma ação e 1000 representa uma cotação de um lote de mil ações.

No arquivo, para o mercado de opções temos cotaçãos 1, lotes de 1000 e lotes de 1000000 de ações

In [ ]:
stg = df[['CODNEG','TPMERC']]
mask = ((stg['TPMERC'] == 12) | (stg['TPMERC'] == 13) | (stg['TPMERC'] == 70) | (stg['TPMERC'] == 80))
df_cot = stg[mask]

fatCorr = df[['FATCOT']]
fatCorr = np.unique(fatCorr)
fatCorr

### ÚLTIMOS VALORES NEGOCIADOS

In [ ]:
df_filter = df[df.CODNEG == Acao]
df_filter = df_filter[(df_filter['DATPRE'] >= '2015-12-01') & (df_filter['DATPRE'] <= '2018-01-01')]
df_filter.head(5)

### PRÉ-PROCESSAMENTO



Como nosso modelo irá prever o preço da opção no dia seguinte, precisaremos analisar apenas os mercados de opções.

O primeiro passo realizado, foi a criação de um novo Data Frame, utilizando como mascara o tipo de mercados, considerando os códigos de mercado de opções (códigos 12,13,70 e 80).

In [ ]:
#Eliminando os mercados que não serão utilizados em nossa análise.Esses mercados são: LEILÃO, FRACIONARIO e o TERMO

mask = ((df['TPMERC'] == 12) | (df['TPMERC'] == 13) | (df['TPMERC'] == 70) | (df['TPMERC'] == 80))
new_df = df[mask]
new_df.head(2)

In [ ]:
#Eliminando a coluna TIPREG, pois esta possui um valor fixo que não será utilizado na análise

new_df.drop(columns=['TIPREG'], axis = 1, inplace = True)
new_df.head(2)

### DADOS FALTANTES

Não encontramos dados faltantes no nosso Dataset.

In [ ]:
new_df.isnull().sum()

### SELEÇÃO DOS ATRIBUTOS DO MODELO

Para o nosso modelo, iremos utilizar os atributos:

PREABE
PREMAX
PREMIN
PREMED
PREULT
PREOFC
PREOFV

In [ ]:
new_df = new_df[['PREABE','PREMAX','PREMIN','PREMED','PREULT','PREOFC','PREOFV','PREEXE']]
new_df.head(2)

### OUTLIERS

Os outliers são dados que se diferenciam drasticamente de todos os outros, são pontos fora da curva. Em outras palavras, um outlier é um valor que foge da normalidade e que pode (e provavelmente irá) causar anomalias nos resultados obtidos por meio de algoritmos e sistemas de análise. Nós vamos analisar os outliers da coluna target do nosso modelo.

#### Outliers na coluna PREULT

In [ ]:
#Verificando o boxplot da nossa coluna target
new_df.boxplot(['PREULT'])

Podemos ver que temos alguns outliers na coluna.

In [ ]:
#Separando a coluna target
valor =  new_df['PREULT']

#Calculando os quartis e o limite inferior e superior do nosso target
Q1 = valor.quantile(.25)
Q3 = valor.quantile(.75)
IIQ = Q3 - Q1
limite_inferior = Q1 - 1.5 * IIQ
limite_superior = Q3 + 1.5 * IIQ

#Selecionando apenas os valores que estão dentro do limite.
selecao = (valor >=limite_inferior) & (valor <= limite_superior)
dados_new = new_df[selecao]

In [ ]:
#Quantidade de dados com outliers
new_df.shape

In [ ]:
#Quantidade de dados se outliers
dados_new.shape

In [ ]:
#Novo boxplot sem os outliers
dados_new.boxplot(['PREULT'])

### SEPARANDO VARIÁVEIS INDEPENDENTES DO MODELO

In [ ]:
X = dados_new.drop(['PREULT'],axis=1)
y = dados_new.PREULT

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
dados_new.head(2)

### CORRELAÇÃO

Vamos analisar a correlação das variáveis com o nosso target

In [ ]:
features = dados_new.iloc[:,[0,1,2,3,5,6,7]].columns.tolist()
target = dados_new.iloc[:,4].name

In [ ]:
correlations = {}
for f in features:
    data_temp = dados_new[[f,target]]
    x1 = data_temp[f].values
    x2 = data_temp[target].values
    key = f + ' vs ' + target
    correlations[key] = pearsonr(x1,x2)[0]

In [ ]:
data_correlations = pd.DataFrame(correlations, index=['Value']).T
data_correlations.loc[data_correlations['Value'].abs().sort_values(ascending=False).index]

Podemos ver que as 4 primeiras variáveis tem uma alta correlação.

#### RELAÇÃO ENTRE PREMIN vs PREULT

In [ ]:
scatter_plot = plt.scatter(new_df['PREMIN'], new_df['PREULT'], alpha=0.5, 
                           c=new_df['PREULT'])
plt.show()

#### RELAÇÃO ENTRE PREMED vs PREULT

In [ ]:
scatter_plot = plt.scatter(new_df['PREMED'], new_df['PREULT'], alpha=0.5, 
                           c=new_df['PREULT'])
plt.show()

#### RELAÇÃO ENTRE PREABE vs PREULT

In [ ]:
scatter_plot = plt.scatter(new_df['PREABE'], new_df['PREULT'], alpha=0.5, 
                           c=new_df['PREULT'])
plt.show()

#### RELAÇÃO ENTRE PREMAX vs PREULT

In [ ]:
scatter_plot = plt.scatter(new_df['PREMAX'], new_df['PREULT'], alpha=0.5, 
                           c=new_df['PREULT'])
plt.show()

#### RELAÇÃO ENTRE PREOFC vs PREULT

In [ ]:
scatter_plot = plt.scatter(new_df['PREOFC'], new_df['PREULT'], alpha=0.5, 
                           c=new_df['PREULT'])
plt.show()

#### RELAÇÃO ENTRE PREEXE vs PREULT

In [ ]:
scatter_plot = plt.scatter(new_df['PREEXE'], new_df['PREULT'], alpha=0.5, 
                           c=new_df['PREULT'])
plt.show()

#### RELAÇÃO ENTRE PREOFV vs PREULT

In [ ]:
scatter_plot = plt.scatter(new_df['PREOFV'], new_df['PREULT'], alpha=0.5, 
                           c=new_df['PREULT'])
plt.show()

### SEPARANDO O DATASET EM TREINO E TESTE

In [ ]:
# separando os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

MinMaxScaler

Muitos algoritmos de aprendizado de máquina funcionam melhor quando as variáveis estão em uma escala relativamente semelhante e quase distribuídos normalmente. MinMaxScaler é um método de pré-processamento que funciona da seguinte maneira: para cada valor em uma variável, o MinMaxScaler subtrai o valor mínimo da variável e depois divide pelo intervalo. O intervalo é a diferença entre o máximo original e o mínimo original.

In [ ]:
#Aplicando MinMaxScaler
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
X_train = mm_scaler.fit_transform(X_train)
X_test = mm_scaler.transform(X_test)

### PREVENDO O PREÇO DA OPÇÃO

#### REGRESSÃO LINEAR

Regressão Linear gera uma equação para descrever a relação estatística entre uma ou mais variáveis preditoras e a variável resposta. A regressão linear encontra a linha que melhor representa as variáveis de entrada com a variável de saída.

In [ ]:
from sklearn import model_selection
from sklearn import linear_model

model = linear_model.LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
from sklearn import metrics

y_pred_train = model.predict(X_train)
y_pred_test  = model.predict(X_test)

print('Coefficients:', model.coef_)

erro_treino = metrics.mean_squared_error(y_train,y_pred_train)
print('RMSE no treino:', erro_treino)

erro_teste = metrics.mean_squared_error(y_test,y_pred_test)
print('RMSE no teste:', erro_teste)

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % metrics.r2_score(y_test, y_pred_test))

In [ ]:
from sklearn.model_selection import cross_val_score
resultado = cross_val_score(model, X_test, y_test, cv = 10)
print(resultado.mean())

### Existência de Overfitting

> Overfitting ocorre quando o seu modelo se ajusta aos seus dados, ou seja, o modelo serve só para os dados da base que foi utilizada para a sua construção. O que ocorre é que nesse caso o modelo passa em diversos testes de precisão com o conjunto de dados utilizados, porém, não serve para predição. Em outras palavras, como alguns cientistas de dados costumam dizer, o seu modelo aprender os dados da base treino ao invés de aprender o todo e ser capaz de fazer previsões. A grosso modo, isso ocorre quando você possui uma alta complexidade e muitos parâmetros se comparado com a base de desenvolvimento. Nesse caso, o modelo serve somente para os dados utilizados no desenvolvimento e suas previsões serão fracas.

#### Fonte: https://estatsite.com/2016/08/18/overfitting-e-cross-validation/

Na verificação dos resultados, identificamos que, o indíce do erro médio (RMSE) estava muito baixo e que a taxa de acerto do modelo estava acima do esperado. 

Num modelo de regressão, o overfitting pode causar valores errôneos de R2, coeficientes p-values.


#### COMO DETECTAR UM OVERFIT NO MODELO

> Você pode detectar o overfitting, determinando se o seu modelo se ajusta a novos dados e se os dados usados para estimar o modelo. Em estatísticas, chamamos isso de validação cruzada, e geralmente envolve o particionamento de seus dados. 
> 
> Esse passo foi efetuado no momento validação, conforme a célula acima, porém para certificar que não havia de fato um overfitting, utilizamos outros método , com o ajuste do erro médio. Se houvesse uma grande variação entre o valor do erro médio e seu valor ajustado, saberíamos que modelo estava ajustado.

##### Fonte: https://statisticsbyjim.com/regression/overfitting-regression-models/

In [ ]:
adj_sample = new_df

model = sm.ols(formula='PREULT ~ PREMAX + PREMIN + PREMED + PREABE ', data=adj_sample)
fitted1 = model.fit()
print('R2: ', fitted1.rsquared)
print('R2 Adj: ', fitted1.rsquared_adj)

Não houve variação consíderavel entre o erro médio e seu erro ajustado. Desta forma, sabemos que o modelo não sofre de um overfitting, o que nos leva a analisar porque os valores estão tão próximos.

##### Fonte: https://statisticsbyjim.com/regression/interpret-adjusted-r-squared-predicted-r-squared-regression/

### MULTICOLINEARIDADE

Um fenômeno que pode explicar por que o baixo erro pode ser a Multicolinearidade.

Em estatística,a multicolinearidade é um problema comum em regressões. Isso acontece pois as varáveis independentes são altamente correlacionadas. 

Isso é problemático porque pode afetar a estabilidade de nossas estimativas de coeficiente, à medida que fazemos pequenas alterações na especificação do modelo.


##### Fonte: https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html

##### Fonte: https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/

### COMO LIDAR COM A MULTICOLINEARIDADE?

> Remova do modelo as preditoras que são altamente correlacionadas.Como eles fornecem informações redundantes, a remoção de um dos fatores correlacionados geralmente não reduz drasticamente o R-quadrado. 

> Análise de Componentes Principais, que são métodos de regressão que reduzem o número de preditoras a um conjunto menor de componentes não correlacionados.

##### Fonte: https://blog.minitab.com/pt/basta-lidando-com-a-multicolinearidade-na-analise-de-regressao


### LASSO - Least Absolute Shrinkage and Selection Operator

> A formula do LASSO foi primeiramente elaborada por  Robert Tibshirani em 1996. É um método utilizado para duas tarefas principais: regularização e seleção de atributos. 
> 
> O método LASSO coloca uma restrição na soma dos valores absolutos dos parâmetros do modelo, a soma deve ser menor que limite superior. Para fazer isso o método aplica um processo de redução em que penaliza coeficientes das variáveis de regressão diminuindo algumas delas para zero. 
> 
> Durante o processo de seleção, as variáveis que ainda possuem um coeficiente diferente de zero após o processo de redução, são selecionados para fazer parte do modelo. O objetivo de esse processo é para minimizar o erro de previsão.


##### Fonte: https://beta.vu.nl/nl/Images/werkstuk-fonti_tcm235-836234.pdf

Utilizaremos então, o método LASSO existente na biblioteca do scikit learn com cross-validation.
Após regularizar o método, o melhor modelo é selecionado através de validação cruzada.

In [ ]:
from sklearn.linear_model import LassoCV

reg = LassoCV()
reg.fit(X_train, y_train)
print("Melhor alpha LassoCV: %f" % reg.alpha_)
print("Melhor score LassoCV: %f" %reg.score(X_train, y_train))
coef = pd.Series(reg.coef_, index = X.columns)

In [ ]:
print("Variaveis consideradas " + str(sum(coef != 0)))
print("Variaveis eliminadas " + str(sum(coef == 0)))

In [ ]:
imp_coef = coef.sort_values()
imp_coef

Desta forma, podemos ver que o melhor modelo utiliza as varáveis PREOFC e PREMIN, porém mesmo após o ajuste, os valores de acerto continuaram altos.

##### Fonte: https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b
##### Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html

### OUTPUT RESULTADOS TREINAMENTO

In [ ]:
# Output dos resultados - Treinamento

d = {'y_train': y_train, 'y_pred_train': y_pred_train}
y_form = pd.DataFrame(data=d)
#y_form.head(10)

y_form.to_excel("output_reglinear_train.xlsx")

### OUTPUT RESULTADOS TESTE

In [ ]:
# Output dos resultados - Teste

d_reg_teste = {'y_test': y_test, 'y_pred_test': y_pred_test}
y_form = pd.DataFrame(data=d_reg_teste)
#y_form.head(10)

y_form.to_excel("output_reglinear_teste.xlsx")

### GRÁFICO VARIAÇÃO ENTRE O REAL E O PREVISTO

In [ ]:
data1 = d_reg_teste['y_test']
data2 = d_reg_teste['y_pred_test']
x = 10*np.array(range(len(data1)))

plt.plot( x, data1, 'go') # green bolinha
plt.plot( x, data1, 'k:', color='orange') # linha pontilha orange

plt.plot( x, data2, 'ro') # red bolinha
plt.plot( x, data2, 'k--', color='blue')  # linha tracejada azul

plt.axis([10, 100, 0, 11])
plt.title("Real X Predição")

plt.grid(False)
plt.xlabel("preço opção")
plt.show()

### REGRESSÃO LOGÍSTICA

A regressão logística é uma técnica estatística que tem como objetivo produzir, a partir de um conjunto de observações, um modelo que permita a predição de valores tomados por uma variável categórica, frequentemente binária, a partir de uma série de variáveis explicativas contínuas e/ou binárias. Em comparação com as técnicas conhecidas em regressão, em especial a regressão linear, a regressão logística distingue-se essencialmente pelo facto de a variável resposta ser categórica.

### REGRESSÃO LOGÍSTICA - XGBOOST

O XGBoost é um algoritmo de aprendizado de máquina de conjunto baseado em árvore de decisão que usa uma estrutura de aumento de gradiente. No XGBoost, as árvores são construídas sequencialmente, de modo que cada árvore subsequente tenha como objetivo reduzir os erros da árvore anterior. Cada árvore aprende com seus antecessores e atualiza os erros residuais. Portanto, a árvore que cresce a seguir na sequência aprenderá com uma versão atualizada dos resíduos.

In [ ]:
xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [ ]:
xgb.fit(X_train,y_train)

In [ ]:
predictions = xgb.predict(X_test)
print(explained_variance_score(predictions,y_test))

### OUTPUT DADOS DE TESTE

In [ ]:
d2 = {'y_test': y_test, 'y_pred_test': predictions}
df_fim = pd.DataFrame(d2)
df_fim.to_excel("teste_xgb.xlsx")

In [ ]:
df_fim.head(5)

### Real X Predição

In [ ]:
data1 = df_fim['y_test']
data2 = df_fim['y_pred_test']
x = 10*np.array(range(len(data1)))

plt.plot( x, data1, 'go') # green bolinha
plt.plot( x, data1, 'k:', color='orange') # linha pontilha orange

plt.plot( x, data2, 'ro') # red bolinha
plt.plot( x, data2, 'k--', color='blue')  # linha tracejada azul

plt.axis([10, 100, 0, 11])
plt.title("Real X Predição")

plt.grid(False)
plt.xlabel("preço opção")
plt.show()

### PREÇO DA OPÇÃO AMANHÃ

In [ ]:
valor = df_fim['y_pred_test'].head(1).mul(100, fill_value=0).to_string(index=False)

In [ ]:
print("O valor da Ação " + Acao + " amanhã será de R$ " + valor) 